In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import pydicom 
from pydicom import dcmread
import matplotlib.pyplot as plt
from cv2 import cv2
from PIL import Image
import os
import pandas as pd
# import time
import zipfile
from tqdm import tqdm

data_zip = zipfile.ZipFile('data.zip', 'w')
# os.mkdir('teste')

In [ ]:
def return_transform_img(ds, window_center, window_width):
    img = ds.pixel_array
    ds.pixel_array
#     print(img.shape)
    intercept = ds[('0028','1052')].value
    slope = ds[('0028','1053')].value

    img = (img*slope +intercept)
    img_min = window_center - window_width//2
    img_max = window_center + window_width//2
    img[img<img_min] = img_min
    img[img>img_max] = img_max

    # img[img<img_min] = 0
    # img[img>img_max] = 255

    img = (img - img_min) / (img_max - img_min)

    return img

train_folder = 'train'

dicom_names = pd.read_csv('../input/testinho/train_dicom_names.csv').values

if not os.path.exists(train_folder):
    os.mkdir(train_folder)

In [ ]:
for name in tqdm(dicom_names[350000:380000]):
#     print(name[0])
#     time.sleep(.300)
#     os.system('kaggle competitions download -c rsna-intracranial-hemorrhage-detection -f rsna-intracranial-hemorrhage-detection/stage_2_train/' + str(name[0]) + '.dcm')
    # kaggle competitions download -c rsna-intracranial-hemorrhage-detection -f rsna-intracranial-hemorrhage-detection/stage_2_train/ID_0000f1657.dcm

    fileName = name[0]
    ds = pydicom.read_file('../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/' + fileName + ".dcm") # read dicom image

    img_subdural = return_transform_img(ds, 80, 200)
    img_brain = return_transform_img(ds, 40, 80)
    img_bone = return_transform_img(ds, 600, 2800)

    cmap = plt.cm.gray

    if not os.path.exists(train_folder + '/' +fileName):
        os.mkdir(train_folder + '/' +fileName)

    # norm = plt.Normalize(vmin=data.min(), vmax=data.max())
    image = cmap(img_subdural)
    plt.imsave('test.jpg', image)
    img_pil = Image.open('test.jpg')
    img = img_pil.resize((224,224), Image.ANTIALIAS)
    img.save(train_folder + '/' +fileName + '/subdural.jpg')
    data_zip.write(train_folder + '/' +fileName + '/subdural.jpg', compress_type=zipfile.ZIP_DEFLATED)

    image = cmap(img_brain)
    plt.imsave('test.jpg', image)
    img_pil = Image.open('test.jpg')
    img = img_pil.resize((224,224), Image.ANTIALIAS)
    img.save(train_folder + '/' + fileName + '/brain.jpg')
    data_zip.write(train_folder + '/' +fileName + '/brain.jpg', compress_type=zipfile.ZIP_DEFLATED)
    

    image = cmap(img_bone)
    plt.imsave('test.jpg', image)
    img_pil = Image.open('test.jpg')
    img = img_pil.resize((224,224), Image.ANTIALIAS)
    img.save(train_folder + '/' +fileName + '/bone.jpg')
    data_zip.write(train_folder + '/' + fileName + '/bone.jpg', compress_type=zipfile.ZIP_DEFLATED)
    
    os.system('rm ' + fileName + '.dcm')

In [ ]:
data_zip.close()








In [ ]:
import os
os.chdir(r'/kaggle/working')



In [ ]:
from IPython.display import FileLink
FileLink(r'data.zip')

In [ ]:
import requests
r = requests.get(FileLink(r'data.zip'), allow_redirects=True)

In [ ]:
open('data.zip', 'wb').write(r.content)

In [ ]:
display(FileLink(r'data.zip'))